# Classification of Pneumonia from Lung Scans

This project is an example of image classification - each input instance is one image, and the output is a categorical label.

As example, we will perform classification on scans of patients' lungs into normal or pneumonia. The complete dataset is available at https://www.kaggle.com/datasets/tawsifurrahman/covid19-radiography-database however is very big. For demonstration purpose, we will only build a model for about 2000 images, half normal and half pneumonia. The data also has xrays of lungs diagnosed with COVID-19 and lung-opacity, however, we will not use them.

This notebook can be used for any image classification task, just make sure that you have the same data organization:
- Images in the same class are stored in a same folder
- The folders' names are the labels
- The data is zipped into a single file

# Load data

Like usual, we first connect our session to Google Drive. Make sure to upload the **lung images.zip** (available on D2L) to a folder there.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Next, we need to unzip the data. We can use the `unzip` command, however, this is an operating system command, not Python, so we need to add `!` like below. Following `!unzip` is the path to the zipped data. After unzipping, verify that you have the new folder created.

In [ ]:
!unzip '/content/drive/MyDrive/IT7133/Week 4/lung images.zip'

# Process data

Like any types of data, image data also needs processing. The cell below performs all the necessary processing for you, including
- standardize the images. Normal pixel values are from `0` to `255`. Standardization transforms them to be between `0` and `1` which is more preferrable by neural networks
- random crop, rotate, and zoom. These processes generate more training data and make our models more robust

However, you do not have to worry too much about the codes in this cell. The images should be processed automatically and get ready for modeling. There will be a lot of outputs from this cell since we also install a few Python packages.

In [ ]:
!pip install datasets evaluate transformers

import PIL, datasets, evaluate
from os import listdir
from os.path import isfile, join
from torchvision.datasets import ImageFolder
from datasets import load_dataset

dataset = load_dataset("imagefolder", data_dir="lung images/")
dataset = dataset['train'].train_test_split(test_size=0.3)
labels = dataset["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

from transformers import AutoImageProcessor
checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

from tensorflow import keras
from keras import layers
import numpy as np
import tensorflow as tf
from PIL import Image
from transformers import DefaultDataCollator
import evaluate
import numpy as np

size = (image_processor.size["height"], image_processor.size["width"])
train_data_augmentation = keras.Sequential(
    [
        layers.RandomCrop(size[0], size[1]),
        layers.Rescaling(scale=1.0 / 127.5, offset=-1),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(height_factor=0.2, width_factor=0.2),
    ],
    name="train_data_augmentation",
)
val_data_augmentation = keras.Sequential(
    [
        layers.CenterCrop(size[0], size[1]),
        layers.Rescaling(scale=1.0 / 127.5, offset=-1),
    ],
    name="val_data_augmentation",
)

def convert_to_tf_tensor(image: Image):
    np_image = np.array(image)
    tf_image = tf.convert_to_tensor(np_image)
    return tf.expand_dims(tf_image, 0)

def preprocess_train(example_batch):
    images = [
        train_data_augmentation(convert_to_tf_tensor(image.convert("RGB"))) for image in example_batch["image"]
    ]
    example_batch["pixel_values"] = [tf.transpose(tf.squeeze(image)) for image in images]
    return example_batch

def preprocess_val(example_batch):
    images = [
        val_data_augmentation(convert_to_tf_tensor(image.convert("RGB"))) for image in example_batch["image"]
    ]
    example_batch["pixel_values"] = [tf.transpose(tf.squeeze(image)) for image in images]
    return example_batch

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

accuracy = evaluate.load("accuracy")
dataset["train"].set_transform(preprocess_train)
dataset["test"].set_transform(preprocess_val)
data_collator = DefaultDataCollator(return_tensors="tf")

Resolving data files:   0%|          | 0/2048 [00:00<?, ?it/s]

# Modeling

In this phase, we build a neural network for the image classification task. We will use an auto model -- the library selects the architecture for us. So, we just need to set a few parameters:
- `num_epochs`: like in the previous module, this is the number of iteration
- `learning_rate`: how fast the model will update in each iteration

In [ ]:
num_epochs = 3
learning_rate = 3e-5

In [ ]:
from transformers import create_optimizer, TFAutoModelForImageClassification
from keras.losses import SparseCategoricalCrossentropy
from transformers.keras_callbacks import KerasMetricCallback

batch_size = 32
num_train_steps = len(dataset["train"]) * num_epochs
weight_decay_rate = 0.01

optimizer, lr_schedule = create_optimizer(
    init_lr=learning_rate,
    num_train_steps=num_train_steps,
    weight_decay_rate=weight_decay_rate,
    num_warmup_steps=0,
)

model = TFAutoModelForImageClassification.from_pretrained(
    checkpoint,
    id2label=id2label,
    label2id=label2id,
)

tf_train_dataset = dataset["train"].to_tf_dataset(
    columns="pixel_values", label_cols="label", shuffle=True, batch_size=batch_size, collate_fn=data_collator
)

tf_eval_dataset = dataset["test"].to_tf_dataset(
    columns="pixel_values", label_cols="label", shuffle=True, batch_size=batch_size, collate_fn=data_collator
)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss)
metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_eval_dataset)
callbacks = [metric_callback]
model.fit(tf_train_dataset, validation_data=tf_eval_dataset, epochs=num_epochs, callbacks=callbacks)

Some layers from the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing TFViTForImageClassification: ['vit/pooler/dense/bias:0', 'vit/pooler/dense/kernel:0']
- This IS expected if you are initializing TFViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3
45/45 [==============================] - 169s 3s/step - loss: 0.4082 - val_loss: 0.2658 - accuracy: 0.9138
Epoch 2/3
45/45 [==============================] - 137s 3s/step - loss: 0.1877 - val_loss: 0.1697 - accuracy: 0.9415
Epoch 3/3
45/45 [==============================] - 147s 3s/step - loss: 0.1618 - val_loss: 0.1299 - accuracy: 0.9593


# Save the Model

Like in the previously, if we like the model, we will save it for deployment.

In [ ]:
model.save_pretrained("/content/drive/MyDrive/IT7133/Week 4/lung_xray_model")